How to create a Deployment
==========================

In this notebook, we show you how to create a Deployment with 3 ReplicaSets. These ReplicaSets are owned by the Deployment and are managed by the Deployment controller. We would also learn how to carry out RollingUpdate and RollBack to new and older versions of the deployment.

In [ ]:
from kubernetes import client, config

### Load config from default location

In [ ]:
config.load_kube_config()
apps_api = client.AppsV1Api()

### Create Deployment object

In [ ]:
deployment = client.V1Deployment()

### Fill required Deployment fields (apiVersion, kind, and metadata)

In [ ]:
deployment.api_version = "apps/v1"
deployment.kind = "Deployment"
deployment.metadata = client.V1ObjectMeta(name="nginx-deployment")

### A Deployment also needs a .spec section

In [ ]:
spec = client.V1DeploymentSpec()
spec.replicas = 3

### Add Pod template in .spec.template section

In [ ]:
spec.template = client.V1PodTemplateSpec()
spec.template.metadata = client.V1ObjectMeta(labels={"app": "nginx"})
spec.template.spec = client.V1PodSpec()

### Pod template container description

In [ ]:
container = client.V1Container()
container.name="nginx"
container.image="nginx:1.7.9"
container. ports = [client.V1ContainerPort(container_port=80)]

In [ ]:
spec.template.spec.containers = [container]
deployment.spec = spec

### Create Deployment

In [ ]:
apps_api.create_namespaced_deployment(namespace="default", body=deployment)

### Update container image

In [ ]:
deployment.spec.template.spec.containers[0].image = "nginx:1.9.1"

### Apply update (RollingUpdate)

In [ ]:
apps_api.replace_namespaced_deployment(name="nginx-deployment", namespace="default", body=deployment)


### Delete Deployment

In [ ]:
apps_api.delete_namespaced_deployment(name="nginx-deployment", namespace="default", body=client.V1DeleteOptions(propagation_policy="Foreground", grace_period_seconds=5))